In [110]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import random as rd
from matplotlib import pyplot as plt
import seaborn as sns
import time
%matplotlib inline
from sklearn.cluster import KMeans
from matplotlib.colors import ListedColormap

Here I created new data frame and scraped from Goldin website which known well for its auctions of trading cards.

In [3]:
#df = pd.DataFrame()

In [6]:
#Single cards - Hockey 
link = "https://goldin.co/buy/?show_only=Sold%20Items&page=1&number_of_lots=96&Item_Type=Single%2520Cards&Sub_Category=Hockey"
driver =webdriver.Chrome()
driver.get(link)
driver.implicitly_wait(10)
cards = driver.find_elements(By.CSS_SELECTOR,".css-175oi2r.r-1udh08x.r-y46g1k")



In [7]:
count1 = 0
count2 = 1

for i in range(15,17):
    time.sleep(2)
    for card in cards:
        driver.implicitly_wait(30)
        title = card.find_element(By.CSS_SELECTOR,".css-1rynq56.r-8akbws.r-krxsd3.r-dnmrzs.r-1udh08x.r-1udbk01").text
        bids = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1d09ksm.r-18u37iz").find_elements(By.CSS_SELECTOR,".css-1rynq56")[1].text
        link = card.find_element(By.CSS_SELECTOR, ".css-175oi2r.r-1awozwy.r-1jj8364.r-13qz1uu").get_attribute("href")
        date = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1wtj0ep.r-kzbkwu.r-m2pi6t.r-1hvjb8t.r-tskmnb").text
        price = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1d09ksm.r-18u37iz").find_elements(By.CSS_SELECTOR,".css-1rynq56")[0].text
        err_date = 0
        err_bids = 0
        try:
            bids = re.findall("([0-9]*) bids", bids)[0]
        except:
            print("Try again in bids...")
            bids = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1d09ksm.r-18u37iz").find_elements(By.CSS_SELECTOR,".css-1rynq56")[1].text
            try:
                bids = re.findall("([0-9]*) bids", bids)[0]
            except:
                err_bids = 1
                bids = str(count1) + "," +str(count2) 

        fixed_date = re.findall("(Mon|Tue|Wed|Thu|Fri|Sat|Sun), (.*), (.*) (AM|PM)", date)
        try:
            weekday = fixed_date[0][0]
            fulldate = fixed_date[0][1]
            hour = fixed_date[0][2]
            ampm = fixed_date[0][3]
        except:
            print("try again in date...")
            try:
                date = card.find_element(By.CSS_SELECTOR,".css-175oi2r.r-1wtj0ep.r-kzbkwu.r-m2pi6t.r-1hvjb8t.r-tskmnb").text
                fixed_date = re.findall("(Mon|Tue|Wed|Thu|Fri|Sat|Sun), (.*), (.*) (AM|PM)", date)
                weekday = fixed_date[0][0]
                fulldate = fixed_date[0][1]
                hour = fixed_date[0][2]
                ampm = fixed_date[0][3]
            except:
                err_date = 1
                weekday = str(count1) + ',' + str(count2)
                fulldate = str(count1) + ',' + str(count2)
                hour = str(count1) + ',' + str(count2)
                ampm = str(count1) + ',' + str(count2)

        '''
        print(title)
        print(bids)
        print(link)
        print(price)
        print(weekday, ", ", fulldate, ", ", hour," ", ampm)
        print("======")
        '''
        
        
        row = pd.Series({"title":title,"bids":bids,"err_bids":err_bids,"link": link, "price": price,"weekday":weekday, "date":fulldate,"err_date":err_date, "hour":hour, "ampm": ampm})
        df = pd.concat([df, row.to_frame().T], ignore_index=True)
        count1 = count1 +1
    link = "https://goldin.co/buy/?show_only=Sold%20Items&page=" +  str(i) +"&number_of_lots=96&Item_Type=Single%2520Cards&Sub_Category=Hockey"
    driver.get(link)
    driver.implicitly_wait(30)
    cards = driver.find_elements(By.CSS_SELECTOR,".css-175oi2r.r-1udh08x.r-y46g1k")
    count2 = count2 + 1
    print("count1: " , count1, " count2: " ,  count2 , " in 15")


count1:  96  count2:  2  in 15
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
Try again in bids...
count1:  192  count2:  3  in 15


Dropping duplicates because I did scraping again and again until I got enough records

In [10]:

#df.drop_duplicates(subset='link', inplace=True)
#df.drop_duplicates(subset='title', inplace=True)
df

,title,bids,err_bids,link,price,weekday,date,err_date,hour,ampm
0,1979-80 O-Pee-Chee #18 Wayne Gretzky Rookie Ca...,10,0,https://goldin.co/item/1979-80-o-pee-chee-18-w...,"$114,000",Thu,5/18/23,0,4:44,AM
1,1966-67 Topps #35 Bobby Orr Rookie Card – PSA ...,18,0,https://goldin.co/item/1966-67-topps-35-bobby-...,"$20,400",Thu,3/16/23,0,3:34,AM
2,1966-67 Topps USA Test #35 Bobby Orr Rookie Ca...,23,0,https://goldin.co/item/1966-67-topps-usa-test-...,"$13,266",Thu,3/16/23,0,4:29,AM
3,1951-52 Parkhurst #66 Gordie Howe Rookie Card ...,23,0,https://goldin.co/item/1951-52-parkhurst-66-go...,"$60,601",Thu,2/09/23,0,4:52,AM
4,2005-06 Upper Deck The Cup Rookie Patch Autogr...,20,0,https://goldin.co/item/2005-06-upper-deck-the-...,"$73,200",Thu,3/30/23,0,4:52,AM
...,...,...,...,...,...,...,...,...,...,...
1435,1984 Topps #96 Pat Lafontaine – PSA EX-MT 6,3,0,https://goldin.co/item/1984-topps-96-pat-lafon...,$11,Fri,5/12/23,0,4:15,AM
1436,1984 O-Pee-Chee #243 Wayne Gretzky,8,0,https://goldin.co/item/1984-o-pee-chee-243-way...,$24,Fri,5/12/23,0,4:15,AM
1437,1973-74 Topps All-Star #150 Bobby Orr - PSA VG 3,2,0,https://goldin.co/item/1973-topps-all-star-150...,$10,Fri,5/05/23,0,4:15,AM
1438,1973-74 Topps All-Star #150 Bobby Orr - PSA EX...,5,0,https://goldin.co/item/1973-topps-all-star-150...,$19,Fri,5/05/23,0,4:15,AM


In [48]:
df = pd.read_csv("Hockey1.csv")
df.drop(df.columns[0], axis=1, inplace=True)
df

,title,bids,err_bids,link,price,weekday,date,err_date,hour,ampm
0,1969 Topps 123 Bob Mccord – PSA NM 7,"191,2",1,https://goldin.co/item/1969-topps-123-bob-mcco...,$6,Fri,5/19/23,0,4:00,AM
1,1973-74 Topps All-Star #150 Bobby Orr - PSA EX...,5,0,https://goldin.co/item/1973-topps-all-star-150...,$19,Fri,5/05/23,0,4:15,AM
2,1973-74 Topps All-Star #150 Bobby Orr - PSA VG 3,2,0,https://goldin.co/item/1973-topps-all-star-150...,$10,Fri,5/05/23,0,4:15,AM
3,1984 O-Pee-Chee #243 Wayne Gretzky,8,0,https://goldin.co/item/1984-o-pee-chee-243-way...,$24,Fri,5/12/23,0,4:15,AM
4,1984 Topps #96 Pat Lafontaine – PSA EX-MT 6,3,0,https://goldin.co/item/1984-topps-96-pat-lafon...,$11,Fri,5/12/23,0,4:15,AM
...,...,...,...,...,...,...,...,...,...,...
1275,2005-06 Upper Deck The Cup Rookie Patch Autogr...,20,0,https://goldin.co/item/2005-06-upper-deck-the-...,"$73,200",Thu,3/30/23,0,4:52,AM
1276,1951-52 Parkhurst #66 Gordie Howe Rookie Card ...,23,0,https://goldin.co/item/1951-52-parkhurst-66-go...,"$60,601",Thu,2/09/23,0,4:52,AM
1277,1966-67 Topps USA Test #35 Bobby Orr Rookie Ca...,23,0,https://goldin.co/item/1966-67-topps-usa-test-...,"$13,266",Thu,3/16/23,0,4:29,AM
1278,1966-67 Topps #35 Bobby Orr Rookie Card – PSA ...,18,0,https://goldin.co/item/1966-67-topps-35-bobby-...,"$20,400",Thu,3/16/23,0,3:34,AM


In [64]:
df = df.sort_index(ignore_index=True)
df

,title,bids,err_bids,link,price,weekday,date,err_date,hour,ampm
0,2017-18 UD The Cup Emblems of Endorsement Auto...,22,0,https://goldin.co/item/2017-18-upper-deck-the-...,"$5,580",Thu,3/30/23,0,5:25,AM
1,2016-17 Upper Deck The Cup Rookie Materials #1...,20,0,https://goldin.co/item/2016-17-upper-deck-the-...,$157,Fri,4/21/23,0,4:24,AM
2,2011-12 Upper Deck Goodwin Champions Autograph...,18,0,https://goldin.co/item/2011-12-upper-deck-good...,$270,Fri,3/24/23,0,4:15,AM
3,2015-16 Upper Deck Young Guns Canvas #C211 Con...,30,0,https://goldin.co/item/2015-16-upper-deck-canv...,"$1,100",Fri,3/24/23,0,4:15,AM
4,2007-08 Upper Deck Generation Next #GN1 Alexan...,5,0,https://goldin.co/item/2007-upper-deck-generat...,$16,Fri,3/31/23,0,4:15,AM
...,...,...,...,...,...,...,...,...,...,...
1138,1963 Parkhurst #55 Gordie Howe - PSA VG-EX 4,17,0,https://goldin.co/item/1963-parkhurst-55-gordi...,$198,Fri,3/03/23,0,4:15,AM
1139,1993-94 Pinnacle Nifty Fifty #10 Brett Hull – ...,2,0,https://goldin.co/item/1993-pinnacle-nifty-fif...,$8,Fri,3/17/23,0,3:15,AM
1140,1993-94 Pinnacle Nifty Fifty #15 Jeremy Roenic...,2,0,https://goldin.co/item/1993-pinnacle-nifty-fif...,$8,Fri,3/17/23,0,3:15,AM
1141,1996-97 Upper Deck Collector's Choice MVP #UD1...,5,0,https://goldin.co/item/1996-collector-s-choice...,$16,Fri,3/10/23,0,4:26,AM


In [65]:
df.drop_duplicates(subset='link', inplace=True)
df

,title,bids,err_bids,link,price,weekday,date,err_date,hour,ampm
0,2017-18 UD The Cup Emblems of Endorsement Auto...,22,0,https://goldin.co/item/2017-18-upper-deck-the-...,"$5,580",Thu,3/30/23,0,5:25,AM
1,2016-17 Upper Deck The Cup Rookie Materials #1...,20,0,https://goldin.co/item/2016-17-upper-deck-the-...,$157,Fri,4/21/23,0,4:24,AM
2,2011-12 Upper Deck Goodwin Champions Autograph...,18,0,https://goldin.co/item/2011-12-upper-deck-good...,$270,Fri,3/24/23,0,4:15,AM
3,2015-16 Upper Deck Young Guns Canvas #C211 Con...,30,0,https://goldin.co/item/2015-16-upper-deck-canv...,"$1,100",Fri,3/24/23,0,4:15,AM
4,2007-08 Upper Deck Generation Next #GN1 Alexan...,5,0,https://goldin.co/item/2007-upper-deck-generat...,$16,Fri,3/31/23,0,4:15,AM
...,...,...,...,...,...,...,...,...,...,...
1138,1963 Parkhurst #55 Gordie Howe - PSA VG-EX 4,17,0,https://goldin.co/item/1963-parkhurst-55-gordi...,$198,Fri,3/03/23,0,4:15,AM
1139,1993-94 Pinnacle Nifty Fifty #10 Brett Hull – ...,2,0,https://goldin.co/item/1993-pinnacle-nifty-fif...,$8,Fri,3/17/23,0,3:15,AM
1140,1993-94 Pinnacle Nifty Fifty #15 Jeremy Roenic...,2,0,https://goldin.co/item/1993-pinnacle-nifty-fif...,$8,Fri,3/17/23,0,3:15,AM
1141,1996-97 Upper Deck Collector's Choice MVP #UD1...,5,0,https://goldin.co/item/1996-collector-s-choice...,$16,Fri,3/10/23,0,4:26,AM


In [68]:
#df.to_csv("Hockey1.csv")

In [1]:
#df["Description"] = ''


<a id='save'>Save into CSV file</a>

In [80]:
start = 0
end = 1143
index =10
df.to_csv('Hockey1.csv')
df.iloc[start:end,:] 

,title,bids,err_bids,link,price,weekday,date,err_date,hour,ampm,Description
0,2017-18 UD The Cup Emblems of Endorsement Auto...,22,0,https://goldin.co/item/2017-18-upper-deck-the-...,"$5,580",Thu,3/30/23,0,5:25,AM,No Data
1,2016-17 Upper Deck The Cup Rookie Materials #1...,20,0,https://goldin.co/item/2016-17-upper-deck-the-...,$157,Fri,4/21/23,0,4:24,AM,No Data
2,2011-12 Upper Deck Goodwin Champions Autograph...,18,0,https://goldin.co/item/2011-12-upper-deck-good...,$270,Fri,3/24/23,0,4:15,AM,Pittsburgh Penguins star Sidney Crosby is the ...
3,2015-16 Upper Deck Young Guns Canvas #C211 Con...,30,0,https://goldin.co/item/2015-16-upper-deck-canv...,"$1,100",Fri,3/24/23,0,4:15,AM,One would be hard-pressed to find a current NH...
4,2007-08 Upper Deck Generation Next #GN1 Alexan...,5,0,https://goldin.co/item/2007-upper-deck-generat...,$16,Fri,3/31/23,0,4:15,AM,"On Dec. 13, 2022, Washington Capitals star Ale..."
...,...,...,...,...,...,...,...,...,...,...,...
1138,1963 Parkhurst #55 Gordie Howe - PSA VG-EX 4,17,0,https://goldin.co/item/1963-parkhurst-55-gordi...,$198,Fri,3/03/23,0,4:15,AM,No Data
1139,1993-94 Pinnacle Nifty Fifty #10 Brett Hull – ...,2,0,https://goldin.co/item/1993-pinnacle-nifty-fif...,$8,Fri,3/17/23,0,3:15,AM,No Data
1140,1993-94 Pinnacle Nifty Fifty #15 Jeremy Roenic...,2,0,https://goldin.co/item/1993-pinnacle-nifty-fif...,$8,Fri,3/17/23,0,3:15,AM,No Data
1141,1996-97 Upper Deck Collector's Choice MVP #UD1...,5,0,https://goldin.co/item/1996-collector-s-choice...,$16,Fri,3/10/23,0,4:26,AM,No Data


In [81]:
long = len(df)
driver =webdriver.Chrome()
for i in range(start,end):
    if(pd.isnull(df.iloc[i,index]) or df.iloc[i,index]=='NaN' or df.iloc[i,index]==""):
        print(i ," in ", long)
        flag = 1
        try:
            link = df.at[i,"link"]
        except:
            print("Error in ", i)
            flag = 0
        if(flag):
            driver.get(link)
            driver.implicitly_wait(8)
        try:
            des = driver.find_element(By.XPATH,"//div[@class='css-175oi2r'][@id = 'html-description']").text
        except:
            des = 'No Data'
        df.iloc[i,index]=des
print("END")

END


For saveing df into CSV file, I need to do again [here](#save)